In [7]:
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, StandardScaler
import src.util as util
import yaml
import copy
from tqdm import tqdm
import os
from datetime import datetime
import openpyxl
from transformers import BertTokenizer, BertModel
import torch

In [8]:
config_data=util.config_load()

In [9]:
def read_Med_code_data(config: dict,filetype) -> pd.DataFrame:
    # Create variable to store raw dataset
    ICD_code_data = pd.DataFrame()

    # Raw Dataset Dir
    raw_dataset_dir = config["raw_dataset_dir_med_codes"]

     # List files in the directory and filter those with filetype in the filename
    files = [f for f in os.listdir(raw_dataset_dir) if filetype in f and f.endswith('.xlsx')]

    # Process and concatenate each filtered file
    for i in tqdm(files):
        file_path = os.path.join(raw_dataset_dir, i)
        ICD_code_data = pd.concat([ICD_code_data, pd.read_excel(file_path)], ignore_index=True,axis=0)


    # Return the concatenated DataFrame
    return ICD_code_data

In [10]:
ICD_10_data=read_Med_code_data(config_data,'ICD10')
ICD_9_old=read_Med_code_data(config_data,'ICD9_old')
ICD_9_new=read_Med_code_data(config_data,'ICD9_new_Jan2024')
ICD_V_data=read_Med_code_data(config_data,'V_codes')
DRG_Code=read_Med_code_data(config_data,'DRG')



100%|██████████| 1/1 [00:00<00:00, 29.00it/s]


In [9]:
PROC_code=pd.read_csv('data/dataset/PRC_code.csv',header=None)

FileNotFoundError: [Errno 2] No such file or directory: 'data/dataset/PRC_code.csv'

In [11]:
ICD_10_data

,CODE,SHORT DESCRIPTION (VALID ICD-10 FY2024),LONG DESCRIPTION (VALID ICD-10 FY2024),NF EXCL
0,A000,"Cholera due to Vibrio cholerae 01, biovar chol...","Cholera due to Vibrio cholerae 01, biovar chol...",NaN
1,A001,"Cholera due to Vibrio cholerae 01, biovar eltor","Cholera due to Vibrio cholerae 01, biovar eltor",NaN
2,A009,"Cholera, unspecified","Cholera, unspecified",NaN
3,A0100,"Typhoid fever, unspecified","Typhoid fever, unspecified",NaN
4,A0101,Typhoid meningitis,Typhoid meningitis,NaN
...,...,...,...,...
73196,Y990,Civilian activity done for income or pay,Civilian activity done for income or pay,Y
73197,Y991,Military activity,Military activity,Y
73198,Y992,Volunteer activity,Volunteer activity,Y
73199,Y998,Other external cause status,Other external cause status,Y


In [21]:
ICD_V_data

,Complete Data,Code,Description
0,V01 Contact with or exposure to communicable d...,V01,Contact with or exposure to communicable dise...
1,V02 Carrier or suspected carrier of infectious...,V02,Carrier or suspected carrier of infectious di...
2,V03 Need for prophylactic vaccination and inoc...,V03,Need for prophylactic vaccination and inocula...
3,V04 Need for prophylactic vaccination and inoc...,V04,Need for prophylactic vaccination and inocula...
4,V05 Need for other prophylactic vaccination an...,V05,Need for other prophylactic vaccination and i...
...,...,...,...
88,v87 Other Specified Personal Exposures And His...,V87,Other Specified Personal Exposures And Histor...
89,v88 Acquired Absence of Other Organs And Tissue,V88,Acquired Absence of Other Organs And Tissue
90,v89 Other Suspected Conditions Not Found,V89,Other Suspected Conditions Not Found
91,v90 Retained Foreign Body,V90,Retained Foreign Body


In [25]:
ICD_9_old

,DIAGNOSIS CODE,LONG DESCRIPTION,SHORT DESCRIPTION
0,0010,Cholera due to vibrio cholerae,Cholera d/t vib cholerae
1,0011,Cholera due to vibrio cholerae el tor,Cholera d/t vib el tor
2,0019,"Cholera, unspecified",Cholera NOS
3,0020,Typhoid fever,Typhoid fever
4,0021,Paratyphoid fever A,Paratyphoid fever a
...,...,...,...
14562,V9129,"Quadruplet gestation, unable to determine numb...",Quad gest-plac/sac undet
14563,V9190,"Other specified multiple gestation, unspecifie...",Mult gest-plac/sac NOS
14564,V9191,"Other specified multiple gestation, with two o...",Mult gest 2+ monochr NEC
14565,V9192,"Other specified multiple gestation, with two o...",Mult gest 2+ monoamn NEC


In [31]:
ICD_9_new

,CODE,LONG DESCRIPTION (VALID ICD-9 FY2024),NF EXCL
0,0010,Cholera due to vibrio cholerae,NaN
1,0011,Cholera due to vibrio cholerae el tor,NaN
2,0019,"Cholera, unspecified",NaN
3,0020,Typhoid fever,NaN
4,0021,Paratyphoid fever A,NaN
...,...,...,...
13517,E9989,Injury due to unspecified war operations but o...,NaN
13518,E9990,Late effect of injury due to war operations,NaN
13519,E9991,Late effect of injury due to terrorism,NaN
13520,NOINJ,NO INJURY,NaN


In [35]:
ICD_9_old[ICD_9_old['DIAGNOSIS CODE']=='73017']['LONG DESCRIPTION'].empty

False

In [36]:
ICD_V_data

,Complete Data,Code,Description
0,V01 Contact with or exposure to communicable d...,V01,Contact with or exposure to communicable dise...
1,V02 Carrier or suspected carrier of infectious...,V02,Carrier or suspected carrier of infectious di...
2,V03 Need for prophylactic vaccination and inoc...,V03,Need for prophylactic vaccination and inocula...
3,V04 Need for prophylactic vaccination and inoc...,V04,Need for prophylactic vaccination and inocula...
4,V05 Need for other prophylactic vaccination an...,V05,Need for other prophylactic vaccination and i...
...,...,...,...
88,v87 Other Specified Personal Exposures And His...,V87,Other Specified Personal Exposures And Histor...
89,v88 Acquired Absence of Other Organs And Tissue,V88,Acquired Absence of Other Organs And Tissue
90,v89 Other Suspected Conditions Not Found,V89,Other Suspected Conditions Not Found
91,v90 Retained Foreign Body,V90,Retained Foreign Body


**ICD Embedding**

In [10]:
ICD_10_data.drop(columns=['SHORT DESCRIPTION (VALID ICD-10 FY2024)','NF EXCL'],inplace=True)
ICD_10_data.rename(columns={'LONG DESCRIPTION (VALID ICD-10 FY2024)':'Description'},inplace=True)
ICD_10_data['File_name']='ICD_10_data'
ICD_10_data['Priority']=1

In [11]:
ICD_9_old.drop(columns=['SHORT DESCRIPTION'],inplace=True)
ICD_9_old.rename(columns={'DIAGNOSIS CODE':'CODE','LONG DESCRIPTION':'Description'},inplace=True)
ICD_9_old['File_name']='ICD_9_old_data'
ICD_9_old['Priority']=3

In [12]:
ICD_9_new.drop(columns=['NF EXCL'],inplace=True)
ICD_9_new.rename(columns={'LONG DESCRIPTION (VALID ICD-9 FY2024)':'Description'},inplace=True)
ICD_9_new['File_name']='ICD_9_new_data'
ICD_9_new['Priority']=2

In [13]:
ICD_9_new

,CODE,Description,File_name,Priority
0,0010,Cholera due to vibrio cholerae,ICD_9_new_data,2
1,0011,Cholera due to vibrio cholerae el tor,ICD_9_new_data,2
2,0019,"Cholera, unspecified",ICD_9_new_data,2
3,0020,Typhoid fever,ICD_9_new_data,2
4,0021,Paratyphoid fever A,ICD_9_new_data,2
...,...,...,...,...
13517,E9989,Injury due to unspecified war operations but o...,ICD_9_new_data,2
13518,E9990,Late effect of injury due to war operations,ICD_9_new_data,2
13519,E9991,Late effect of injury due to terrorism,ICD_9_new_data,2
13520,NOINJ,NO INJURY,ICD_9_new_data,2


In [16]:
merged_df = (
    ICD_9_old
    .merge(ICD_9_new, on=['CODE', 'Description'], how='outer', suffixes=('_old', '_new'))
    .merge(ICD_10_data, on=['CODE', 'Description'], how='outer', suffixes=('', '_10'))
)


merged_df['file_name_combined'] = (
    merged_df[['File_name', 'File_name_new','File_name_old']].bfill(axis=1).iloc[:, 0]
)

merged_df['Priority_combined'] = (
    merged_df[['Priority', 'Priority_new','Priority_old']].bfill(axis=1).iloc[:, 0]
)

# Selecting relevant columns for the final DataFrame
result = merged_df[['CODE', 'Description', 'Priority_combined', 'file_name_combined']]


In [18]:
merged_df

,CODE,Description,File_name_old,Priority_old,File_name_new,Priority_new,File_name,Priority,file_name_combined,Priority_combined
0,0010,Cholera due to vibrio cholerae,ICD_9_old_data,3.0,ICD_9_new_data,2.0,NaN,NaN,ICD_9_new_data,2.0
1,0011,Cholera due to vibrio cholerae el tor,ICD_9_old_data,3.0,ICD_9_new_data,2.0,NaN,NaN,ICD_9_new_data,2.0
2,0019,"Cholera, unspecified",ICD_9_old_data,3.0,ICD_9_new_data,2.0,NaN,NaN,ICD_9_new_data,2.0
3,0020,Typhoid fever,ICD_9_old_data,3.0,ICD_9_new_data,2.0,NaN,NaN,ICD_9_new_data,2.0
4,0021,Paratyphoid fever A,ICD_9_old_data,3.0,ICD_9_new_data,2.0,NaN,NaN,ICD_9_new_data,2.0
...,...,...,...,...,...,...,...,...,...,...
87884,Y990,Civilian activity done for income or pay,NaN,NaN,NaN,NaN,ICD_10_data,1.0,ICD_10_data,1.0
87885,Y991,Military activity,NaN,NaN,NaN,NaN,ICD_10_data,1.0,ICD_10_data,1.0
87886,Y992,Volunteer activity,NaN,NaN,NaN,NaN,ICD_10_data,1.0,ICD_10_data,1.0
87887,Y998,Other external cause status,NaN,NaN,NaN,NaN,ICD_10_data,1.0,ICD_10_data,1.0


In [1]:
result[result.CODE=='4730'].to_csv('data_check.csv')

NameError: name 'result' is not defined

In [12]:
result.sort_values(by='Priority_combined',inplace=True)
data_ICD_deduplicated=result.drop_duplicates(subset='CODE',keep='first')


/var/folders/nd/f34czdrj02175j4j_k03vfyw0000gp/T/ipykernel_11279/1583808023.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.sort_values(by='Priority_combined',inplace=True)


In [13]:
data_ICD_deduplicated=data_ICD_deduplicated[['CODE',	'Description']]

In [14]:
data_ICD_deduplicated.dropna()

,CODE,Description
87875,Y93J1,"Activity, piano playing"
87874,Y93I9,"Activity, other involving external motion"
87873,Y93I1,"Activity, roller coaster riding"
87872,Y93H9,"Activity, other involving exterior property an..."
87840,Y93A6,"Activity, grass drills"
...,...,...
16779,E8495,Street and highway accidents
16780,E8496,Accidents occurring in public building
16781,E8497,Accidents occurring in residential institution
16782,E8498,Accidents occurring in other specified places


In [8]:
def Biobert_data_embeddings (inp_df):
        # Load pre-trained BioBERT model and tokenizer
        tokenizer = BertTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")
        model = BertModel.from_pretrained("dmis-lab/biobert-base-cased-v1.2")

        # Sample DataFrame with a text column
        df = inp_df
        df.dropna(inplace=True)
        # Tokenize the text column
        inputs = tokenizer(df['Description'].tolist(), padding=True, truncation=True, return_tensors="pt")

        # Generate embeddings
        with torch.no_grad():
            outputs = model(**inputs)

        # Extract embeddings for [CLS] token (index 0)
        embeddings = outputs.last_hidden_state[:, 0, :]
        # norm = embeddings.norm(p=2, dim=1, keepdim=True)
        # normalized_embeddings = embeddings.div(norm)

        # Add normalized embeddings to the DataFrame
        #df['normalized_embeddings'] = normalized_embeddings.tolist()


        # Convert tensor to list format
        df['embeddings'] = embeddings.tolist()
        return df


In [17]:
df_combined=pd.DataFrame()
batch_size=10000
for i in range(0,len(data_ICD_deduplicated),batch_size):
    df=data_ICD_deduplicated[i:i+batch_size]
    df_icd_code_embeds=Biobert_data_embeddings(df)
    df_combined=pd.concat([df_combined,df_icd_code_embeds],ignore_index=True)
df_combined.to_excel('ICD_embeddings.xlsx')
util.pickle_dump(df_combined, config_data["raw_dataset_path_ICD_embeddings"])

/Users/Geethika/Documents/GitHub/Medicare Claims/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/var/folders/nd/f34czdrj02175j4j_k03vfyw0000gp/T/ipykernel_11279/3496820813.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/var/folders/nd/f34czdrj02175j4j_k03vfyw0000gp/T/ipykernel_11279/3496820813.py:26: Se

**V_code**

In [26]:
ICD_V_data.drop(columns=['Complete Data'],inplace=True)

In [27]:
df_v_code_embeds=Biobert_data_embeddings(ICD_V_data)

/Users/Geethika/Documents/GitHub/Medicare Claims/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [33]:
df_v_code_embeds.rename(columns={'Code':'CODE'},inplace=True)
util.pickle_dump(df_v_code_embeds, config_data["raw_dataset_path_v_code_embeddings"])


**Combine V code + ICD**

In [49]:
df_ICD_code_embeddings=pd.concat([df_combined,df_v_code_embeds],axis=0)
util.pickle_dump(df_ICD_code_embeddings, config_data["raw_dataset_path_ICD_embeddings_combined"])

**DRG_Code**

In [119]:
DRG_Code['DRG'] = DRG_Code['DRG'].map(lambda x: f'{x:0>3}')

In [120]:
DRG_Code.rename(columns={'DRG':'CODE'},inplace=True)

In [121]:
df_DRG_embeds=Biobert_data_embeddings(DRG_Code)


/Users/Geethika/Documents/GitHub/Medicare Claims/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [122]:
df_DRG_embeds

,CODE,Description,embeddings
0,000,Administrative,"[0.4769437909126282, -0.03629649803042412, -0...."
1,OTH,Others. Miscellaneous,"[0.5168061256408691, -0.12911075353622437, -0...."
2,001,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,"[-0.2817968428134918, -0.1383458375930786, -0...."
3,002,HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SY...,"[-0.30018964409828186, -0.18892642855644226, -..."
4,003,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PDX ...,"[0.29879578948020935, -0.1733267903327942, -0...."
...,...,...,...
761,987,NON-EXTENSIVE O.R. PROC UNRELATED TO PRINCIPAL...,"[0.08985249698162079, -0.2609464228153229, -0...."
762,988,NON-EXTENSIVE O.R. PROC UNRELATED TO PRINCIPAL...,"[0.11839408427476883, -0.27363041043281555, -0..."
763,989,NON-EXTENSIVE O.R. PROCEDURE UNRELATED TO PRIN...,"[0.13221675157546997, -0.07034632563591003, -0..."
764,998,PRINCIPAL DIAGNOSIS,"[0.20965079963207245, -0.38666093349456787, -0..."


In [123]:
util.pickle_dump(df_DRG_embeds, config_data["raw_dataset_path_DRG_embeddings"]) 

***Process Code**

In [89]:
PROC_code

,0
0,0001 Ther ult head & neck ves
1,0002 Ther ultrasound of heart
2,0003 Ther ult peripheral ves
3,0009 Other therapeutic ultsnd
4,0010 Implant chemothera agent
...,...
3877,9995 Stretching of foreskin
3878,9996 Sperm collection
3879,9997 Denture fitting
3880,9998 Milk extraction


In [90]:
PROC_code.rename(columns={0:'Complete_data'},inplace=True)

In [91]:
PROC_code['CODE']=PROC_code['Complete_data'].str[0:4]
PROC_code['Description']=PROC_code['Complete_data'].str[6:len(PROC_code['Complete_data'])]

In [92]:
PROC_code['Description']=PROC_code['Complete_data'].str[5:len(PROC_code['Complete_data'])]

In [94]:
PROC_code.drop(columns=['Complete_data'],inplace=True)
PROC_code['CODE'] = PROC_code['CODE'].map(lambda x: f'{x:0>3}')

In [95]:
PROC_code.dropna(inplace=True)

In [97]:
PROC_code['CODE']=PROC_code['CODE'].str.strip()

In [111]:
PROC_code['CODE'] = PROC_code['CODE'].str.strip().str.zfill(4)

In [116]:
df_PRC_embeds=Biobert_data_embeddings(PROC_code)

/Users/Geethika/Documents/GitHub/Medicare Claims/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [117]:
util.pickle_dump(df_PRC_embeds, config_data["raw_dataset_path_PROC_embeddings"]) 

In [107]:
PROC_code

,CODE,Description,embeddings
0,0001,Ther ult head & neck ves,"[-0.02448565699160099, -0.012898515909910202, ..."
1,0002,Ther ultrasound of heart,"[-0.13837124407291412, -0.11132107675075531, -..."
2,0003,Ther ult peripheral ves,"[-0.005060335621237755, -0.00894838199019432, ..."
3,0009,Other therapeutic ultsnd,"[0.3417268395423889, -0.2001538872718811, -0.1..."
4,0010,Implant chemothera agent,"[-0.012385757640004158, -0.41206079721450806, ..."
...,...,...,...
3877,9995,Stretching of foreskin,"[0.017130233347415924, -0.1918523609638214, -0..."
3878,9996,Sperm collection,"[0.09843242913484573, -0.287685364484787, -0.2..."
3879,9997,Denture fitting,"[0.1860845386981964, -0.20453974604606628, -0...."
3880,9998,Milk extraction,"[0.136044442653656, -0.24214959144592285, -0.1..."


In [ ]:
df_PRC_embeds['CODE']=df_PRC_embeds['CODE'].zfill(4)

In [126]:
df_PRC_embeds['length_of_embeddings'] = df_PRC_embeds['embeddings'].apply(len)